In [1]:
%pushd book-materials
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

# 챕터 기본 설정값은 저자가 생성해둔 깃허브 코드를 참조했다. 
# 링크: https://github.com/Ahnjunhyeok/pydata-book/blob/3rd-edition/ch08.ipynb


[WinError 2] 지정된 파일을 찾을 수 없습니다: 'book-materials'
c:\Users\ahnjh\Desktop\pythonWorkspace2\Jump_to_Python\Python_for_DA


In [2]:
# Chapter 8 데이터 준비하기: 조인, 병합, 변형

In [3]:
# 대부분의 경우 데이터는 여러 파일이나 데이터베이스 혹은 분석하기 쉽지 않은 형태로 기록되어 있다. 

In [4]:
# 이 장에서는 데이터를 합치고, 재배열할 수 있는 도구들을 살펴보자.

In [5]:
# 8.1 계층적 색인

In [6]:
# 계층적 색인은 pandas의 중요한 기능인데 축에 대해 다중(둘 이상) 색인 단계를 지정할 수 있도록 해준다. 

In [7]:
# 약간 추상적으로 말하면, 높은 차원의 데이터를 낮은 차원의 형식으로 다룰 수 있게 해주는 기능이다. 

In [8]:
# 간단한 예제를 하나 살펴보자. 우선 리스트의 리스트(또는 배열)를 색인으로 하는 Series를 하나 생성하자. 

In [9]:
import numpy as np
import pandas as pd
import re
import sqlalchemy as sqla

In [10]:
data = pd.Series(np.random.randn(9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [11]:
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [12]:
# 지금 생성한 객체가 MultiIndex를 색인으로 하는 Series인데, 색인의 계층을 보여주고 있다. 

In [13]:
# 바로 위 단계의 색인을 이용해서 하위 계층을 직접 접근할 수 있다. 

In [14]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [15]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능하다. 

In [16]:
data["b"]

1   -0.555730
3    1.965781
dtype: float64

In [17]:
data["b":"c"]

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [18]:
data.loc[["b", "d"]]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [19]:
# 하위 계층의 객체를 선택하는 것도 가능하다. 

In [20]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [21]:
# 계층적인 색인은 데이터를 재형성하고 피벗테이블 생성 같은 그룹 기반의 작업을 할 때 중요하게 사용된다.

In [22]:
# 예를 들어 위에서 만든 DataFrame 객체에 unstack 메서드를 사용해서 데이터를 새롭게 배열할 수도 있다.

In [23]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [24]:
# unstack의 반대 작업은 stack 메서드로 수행한다.

In [25]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [26]:
# stack과 unstack 메서드는 이 장 후반부에서 더 자세히 알아보기로 하자. 

In [27]:
# DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있다.

In [28]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])

In [29]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [30]:
# 계층적 색인의 각 단계는 이름(문자열이나 어떤 파이썬 객체라도 가능하다)을 가질 수 있고, 만약 이름을 가지고 있다면 콘솔 출력 시 함께 나타난다.

In [31]:
frame.index.names = ["key1", "key2"]

In [32]:
frame.columns.names = ["state", "color"]

In [33]:
frame # Caution! 색인 이름인 "state"와 "color"를 로우 라벨과 혼동하지 말자.

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [34]:
frame.index.nlevels

2

In [35]:
# 컬럼의 부분집합을 부분적인 색인으로 접근하는 것도 컬럼에 대한 부분적 색인과 비슷하게 사용하면 된다.

In [36]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [37]:
# MultiIndex는 따로 생성한 다음에 재사용 가능하다. 위에서 살펴본 DataFrame의 컬럼 계층 이름은 다음처럼 생성할 수 있다.

In [38]:
# MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"], ["Green", "Red", "Green"]],
#                         names=["State", "color"])

In [39]:
# 8.1.1 계층의 순서를 바꾸고 정렬하기

In [40]:
# 계층적 색인에서 계층의 순서를 바꾸거나 지정된 계층에 따라 데이터를 정렬해야 하는 경우가 있을 수 있다.

In [41]:
# swaplevel은 넘겨받은 두 개의 계층 번호나 이름이 뒤바뀐 새로운 객체를 반환한다(하지만 데이터는 변경되지 않는다)

In [42]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [43]:
# 반면 sort_index 메서드는 단일 계층에 속한 데이터를 정렬한다. 

In [44]:
# swaplevel을 이용해서 계층을 바꿀 때 sort_index를 사용해서 결과가 사전적으로 정렬되도록 만드는 것도 드물지 않은 일이다. 

In [45]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [46]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [47]:
# Note_ 객체가 계층적 색인으로 상위 계층부터 사전식으로 정렬되어 있다면(sort_index(level=0)이나 sort_index()의 결과처럼) 데이터를 선택하는 성능이 훨씬 좋아진다.

In [48]:
# 8.1.2 계층별 요약 통계

In [49]:
# DataFrame과 Series의 많은 기술 통계와 요약 통계는 level 옵션을 가지고 있는데, 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션이다.

In [50]:
# 앞에서 살펴본 DataFrame에서 로우나 컬럼을 아래처럼 계층별로 합할 수 있다.

In [51]:
frame.sum(level="key2")

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_22160\3430452342.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level="key2")


state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [52]:
frame.sum(level="color", axis=1)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_22160\3066880672.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  frame.sum(level="color", axis=1)


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [53]:
# 이는 내부적으로 pandas의 groupby 기능을 이용해서 구현되었는데, 자세한 내용은 앞으로 더 살펴보겠다.

In [54]:
# 8.1.3 DataFrame의 컬럼 사용하기

In [55]:
# DataFrame에서 로우를 선택하기 위한 색인으로 하나 이상의 컬럼을 사용하는 것은 드물지 않은 일이다. 

In [56]:
# 아니면 로우의 색인을 DataFrame의 컬럼으로 옮기고 싶을 것이다. 다음은 예제 DataFrame이다.

In [57]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})

In [58]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [59]:
# DataFrame의 set_index 함수는 하나 이상의 컬럼을 색인으로 하는 새로운 DataFrame을 생성한다. 

In [60]:
frame2 = frame.set_index(["c", "d"])

In [61]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [62]:
# 다음처럼 컬럼을 명시적으로 남겨두지 않으면 DataFrame에서 삭제된다.

In [63]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [64]:
# 반면 reset_index 함수는 set_index와 반대되는 개념인데 계층적 색인 단계가 컬럼으로 이동한다. 

In [65]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


In [66]:
# 8.2 데이터 합치기

In [67]:
# pandas 객체에 저장된 데이터는 여러 가지 방법으로 합칠 수 있다.

In [68]:
# 1. pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터베이스의 join 연산과 유사하다.
# 2. pandas.concat은 하나의 축을 따라 객체를 이어붙인다.
# 3. combile_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다. 

In [69]:
# 각각의 데이터를 합치는 방법에 대해 다양한 예제와 함께 살펴보게 될 것이다. 이 기법은 책 전체에서 계속 활용된다.

In [70]:
# 8.2.1 데이터베이스 스타일로 DataFrame 합치기

In [71]:
# merge(병합)나 join(조인) 연산은 관계형 데이터베이스의 핵심적인 연산인데, 하나 이상의 키를 사용해서 데이터 집합의 로우를 합친다.

In [72]:
# pandas의 merge 함수를 이용해서 이런 알고리즘을 데이터에 적용할 수 있다. 예제를 살펴보자

In [73]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})

In [74]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": range(3)})

In [75]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [76]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [77]:
# 위 예제는 '다대일'의 경우다. df1의 데이터는 key 컬럼에 여러 개의 a, b 로우를 가지고 있고 df2의 데이터는 key 컬럼에 유일한 로우를 가지고 있다.

In [78]:
# 이 객체에 대해 merge 함수를 호출하면 다음과 같은 결과를 얻는다.

In [79]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [80]:
# 위에서 나는 어떤 컬럼을 병합할 것인지 명시하지 않았는데, merge 함수는 중복된 컬럼 이름을 키로 사용한다(위 예에서는 key 컬럼).

In [81]:
# 하지만 명시적으로 지정해주는 습관을 들이는 게 좋다.

In [82]:
pd.merge(df1, df2, on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [83]:
# 만약 두 객체에 중복된 컬럼 이름이 하나도 없다면 따로 지정해주면 된다.

In [84]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": range(7)})

In [85]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": range(3)})

In [86]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [87]:
# 결과를 잘 살펴보면 "c"와 "d"에 해당하는 값이 빠진 것을 알 수 있다. 

In [88]:
# merge 함수는 기본적으로 내부 조인(inner join)을 수행하여 교집합인 결과를 반환한다. 

In [89]:
# how 인자로 "left", "right", "outer"를 넘겨서 각각 왼쪽 조인, 오른쪽 조인, 외부 조인을 수행할 수도 있다. 

In [90]:
# 외부 조인은 합집합인 결과를 반환하고 왼쪽 조인과 오른쪽 조인은 각각 왼쪽 또는 오른쪽의 모든 로우를 포함하는 결과를 반환한다. 

In [91]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [92]:
# [표 8-1]에 how 옵션에 따라 조인 연산이 어떻게 동작하는지 요약해두었다. 페이지 317

In [93]:
# "다대다" 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다. 

In [94]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": range(6)})

In [95]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": range(5)})

In [96]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [97]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [98]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [99]:
# 다대다 조인은 두 로우의 데카르트곱을 반환한다. 

In [100]:
# 왼쪽 DataFrame에는 3개의 "b" 로우가 있고 오른쪽에는 2개의 "b" 로우가 있으며, 결과는 6개의 "b" 로우가 된다. 

In [101]:
# 조인 메서드는 결과에 나타나는 구별되는 키에 대해서만 적용된다. 

In [102]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [103]:
# 여러 개의 키를 병합하려면 컬럼 이름이 담긴 리스트를 넘기면 된다. 

In [104]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": [1, 2, 3]})

In [105]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": [4, 5, 6, 7]})

In [106]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [107]:
# merge 메서드의 종류에 따라 어떤 키 조합이 결과로 반환되는지 알려면 실제 구현과는 조금 다르지만 여러 개의 키가 들어 있는 튜플의 배열이 단일 조인키로 사용된다고 생각하면 된다. 

In [108]:
# Caution! 컬럼과 컬럼을 조인할 때 DataFrame 객체의 색인은 무시된다. 

In [109]:
# 병합 연산에서 고려해야 할 마지막 사항은 겹치는 컬럼 이름에 대한 처리다. 

In [110]:
# 다음에 살펴보겠지만 축 이름을 변경해서 수동으로 컬럼 이름이 겹치게 할 수도 있고, merge 함수에 있는 suffixes 인자로 두 DataFrame 객체에서 겹치는 컬럼 이름 뒤에 붙일 문자열을 지정해줄 수도 있다.

In [111]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [112]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [113]:
# [표 8-2]에 merge 함수의 인자를 나타냈다. 다음 절에서는 DataFrame의 로우 색인에 대한 조인을 알아본다. 페이지 320-321

In [114]:
# 8.2.2 색인 변형하기

In [115]:
# 병합하려는 키가 DataFrame의 색인일 경우가 있다. 

In [116]:
# 이런 경우에는 left_index=True 혹은 right_index=True 옵션(또는 둘 다)을 지정해서 해당 색인을 병합키로 사용할 수 있다. 

In [117]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                     "value": range(6)})

In [118]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

In [119]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [120]:
right1

,group_val
a,3.5
b,7.0


In [121]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [122]:
# 병합은 기본적으로 교집합을 구하지만 외부 조인을 실행해서 합집합을 구할 수도 있다. 

In [123]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [124]:
# 계층 색인된 데이터는 암묵적으로 여러 키를 병합하는 것이라 약간 복잡하다. 

In [125]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": np.arange(5.)})

In [126]:
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[["Nevada", "Nevada", "Ohio", "Ohio",
                              "Ohio", "Ohio"],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                             columns=["event1" ,"event2"])

In [127]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [128]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [129]:
# 이 경우에는 리스트로 여러 개의 컬럼을 지정해서 병합해야 한다(중복되는 색인값을 다룰 때는 how="outer" 옵션을 사용해야 한다).

In [130]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [131]:
pd.merge(lefth, righth, left_on=["key1", "key2"],
         right_index=True, how="outer")

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [132]:
# 양쪽에 공통적으로 존재하는 여러 개의 색인을 병합하는 것도 가능하다.

In [133]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"], 
                     columns=["Ohio", "Nevada"])

In [134]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"])

In [135]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [136]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [137]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [138]:
# 색인으로 병합할 때 DataFrame의 join 메서드를 사용하면 편리하다. 

In [139]:
# join 메서드는 컬럼이 겹치지 않으며 완전히 같거나 유사한 색인 구조를 가진 여러 개의 DataFrame 객체를 병합할 때 사용할 수 있다. 
# 위에서 본 예제는 다음처럼 작성할 수 있다. 

In [140]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [141]:
# 과거에 작성된 pandas의 일부 코드 제약으로 인해 DataFrame의 join 메서드는 왼쪽 조인을 수행한다. 

In [142]:
# join 메서드를 호출한 DataFrame의 컬럼 중 하나에 대해 조인을 수행하는 것도 가능하다. 

In [143]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [144]:
# 마지막으로 색인 대 색인으로 두 DataFrame을 병합하려면 그냥 간단하게 병합하려는 DataFrame의 리스트를 join 메서드로 넘기면 된다. 

In [145]:
# 하지만 보통 이런 병합은 아래에서 살펴볼 concat 메서드를 사용한다. 

In [146]:
another = pd.DataFrame([[7., 8.], [9., 10], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"], 
                       columns=["New York", "Oregon"])

In [147]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [148]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [149]:
left2.join([right2 , another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


In [150]:
# 8.2.3 축 따라 이어붙이기

In [151]:
# 데이터를 합치는 또 다른 방법으로 concatenation(이어붙이기)(binding, stacking(연결, 적층)이라고도 한다)가 있다. 

In [152]:
# NumPy는 ndarray를 이어붙이는 concatenate 함수를 제공한다. 

In [153]:
arr = np.arange(12).reshape((3, 4))

In [154]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [155]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [156]:
# Series나 DataFrame 같은 pandas 객체의 컨텍스트 내부에는 축마다 이름이 있어서 배열을 쉽게 이어붙일 수 있도록 되어 있다. 

In [157]:
# 이때 다음 사항을 고려해야 한다. 

In [158]:
# 1. 만약 연결하려는 두 객체의 색인이 서로 다르면 결과는 그 색인의 교집합이어야 하는가 아니면 합집합이어야 하는가?
# 2. 합쳐진 결과에서 합쳐지기 전 객체의 데이터를 구분할 수 있어야 하는가?
# 3. 어떤 축으로 연결할 것인지 고려해야 하는가? 많은 경우 DataFrame의 기본 정수 라벨이 가장 먼저 무시된다. 

In [159]:
# pandas의 concat 함수는 위 사항에 대한 답을 제공한다. concat 함수가 어떻게 동작하는지 다양한 예제로 알아보자.

In [160]:
# 색인이 겹치지 않는 3개의 SEries 객체가 있다고 하자. 

In [161]:
s1 = pd.Series([0, 1], index=["a", "b"])

In [162]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"])

In [163]:
s3 = pd.Series([5, 6], index=["f", "g"])

In [164]:
#  이 세 객체를 리스트로 묶어서 concat 함수에 전달하면 값과 색인을 연결해준다. 

In [165]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [166]:
# concat 함수는 axis=0을 기본값으로 하여 새로운 Series 객체를 생성한다. 

In [168]:
# 만약 axis=1을 넘긴다면 결과는 Series가 아니라 DataFrame이 될 것이다(axis=1은 컬럼을 의미한다).

In [169]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [170]:
# 겹치는 축이 없기 때문에 외부 조인으로 정렬된 합집합을 얻었지만 join="inner"를 넘겨서 교집합을 구할 수도 있다. 

In [171]:
s4 = pd.concat([s1, s3])

In [172]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [178]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [179]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


In [180]:
# 마지막 예제에서 "f"와 "g" 라벨은 join="inner" 옵션으로 인해 사라지게 된다.

In [181]:
# join_axes 인자로 병합하려는 축을 직접 지정해줄 수도 있다. 

In [210]:
# pd.concat([s1, s4], axis=1, join_axes = [["a", "c", "b", "e"]]) 
# pd.concat([s1, s4], axis=1).reindex("a", "c", "d", "e") # TypeError: Cannot specify all of 'labels', 'index', 'columns'. 발생
# join_axes는 사용이 중단되었다. 대신 reindex를 사용해야 한다. 

In [211]:
# Series를 이어붙이기 전의 개별 Series를 구분할 수 없는 문제가 생기는데, 이어붙인 축에 대해 계층적 색인을 생성하여 식별이 가능하도록 할 수 있다. 

In [212]:
# 계층적 색인을 생성하려면 keys 인자를 사용하면 된다. 

In [213]:
result = pd.concat([s1, s2, s3], keys=["one", "two", "three"])

In [214]:
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [215]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [216]:
# Series를 axis=1로 병합할 경우 keys는 DataFrame의 컬럼 제목이 된다. 

In [217]:
pd.concat([s1, s2, s3], axis=1, keys=["one", "two", "three"])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [218]:
# DataFrame 객체에 대해서도 지금까지와 같은 방식으로 적용할 수 있다. 

In [219]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])

In [220]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])

In [221]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [222]:
df2

,three,four
a,5,6
c,7,8


In [223]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [224]:
# 리스트 대신 객체의 사전을 넘기면 사전의 키가 keys 옵션으로 사용된다. 

In [225]:
pd.concat({"level1": df1, "level2": df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [235]:
# 계층적 색인을 생성할 때 사용할 수 있는 몇 가지 추가적인 옵션은 [표8-3]을 참조하자. 예를 들어 새로 생성된 계층의 이름은 names 인자로 지정할 수 있다. 표 8-3 페이지 331-332

In [236]:
pd.concat([df1, df2], axis=1, keys=["level1", "level2"],
          names=["upper", "lower"])

upper    level1                                  level2                    
lower         a         b         c         d         b         d         a
0      1.246435  1.007189 -1.296221  0.274992  0.476985  3.248944 -1.021228
1      0.228913  1.352917  0.886429 -2.001637 -0.577087  0.124121  0.302614
2     -0.371843  1.669025 -0.438570 -0.539741       NaN       NaN       NaN

In [237]:
# 마지막으로 DataFrame의 로우 색인이 분석에 필요한 데이터를 포함하고 있지 않은 경우 어떻게 할 것인가?

In [238]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=["a", "b", "c", "d"])

In [239]:
df2 = pd.DataFrame(np.random.randn(2, 3), columns=["b", "d", "a"])

In [240]:
df1

,a,b,c,d
0,0.523772,0.000940,1.343810,-0.713544
1,-0.831154,-2.370232,-1.860761,-0.860757
2,0.560145,-1.265934,0.119827,-1.063512


In [241]:
df2

,b,d,a
0,0.332883,-2.359419,-0.199543
1,-1.541996,-0.970736,-1.307030


In [242]:
# 이 경우 ignore_index=True 옵션을 주면 된다.

In [243]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.523772,0.000940,1.343810,-0.713544
1,-0.831154,-2.370232,-1.860761,-0.860757
2,0.560145,-1.265934,0.119827,-1.063512
3,-0.199543,0.332883,NaN,-2.359419
4,-1.307030,-1.541996,NaN,-0.970736


In [244]:
# 8.2.4 겹치는 데이터 합치기

In [245]:
# 데이터를 합칠 때 병합이나 이어붙이기로는 불가능한 상황이 있는데, 두 데이터셋의 색인이 일부 겹치거나 전체가 겹치는 경우가 그렇다.

In [246]:
# 벡터화된 if-else 구문을 표현하는 NumPy의 where 함수로 자세히 알아보자.

In [247]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])

In [248]:
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=["f", "e", "d", "c", "b", "a"])

In [250]:
b[-1] = np.nan

In [251]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [252]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [253]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [254]:
# Series 객체의 combine_fisrt 메서드는 위와 동일한 연산을 제공하며 데이터 정렬 기능까지 제공한다. 

In [255]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [256]:
# DataFrame에서 combine_first 메서드는 컬럼에 대해 같은 동작을 한다. 

In [257]:
# 그러므로 호출하는 객체에서 누락된 데이터를 인자로 넘긴 객체에 있는 값으로 채워 넣을 수 있다. 

In [259]:
df1 = pd.DataFrame({"a":[1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})

In [260]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})

In [261]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [262]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [263]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


In [264]:
# 8.3 재형성과 피벗

In [265]:
# 표 형식의 데이터를 재배치하는 다양한 기본 연산이 존재한다. 이런 연산을 재형성 또는 피벗 연산이라고 한다. 

In [266]:
# 8.3.1 계층적 색인으로 재형성하기

In [267]:
# 계층적 색인은 DataFrame의 데이터를 재배치하는 다음과 같은 방식을 제공한다. 

In [269]:
# 1. stack: 데이터의 컬럼을 로우로 피벗(또는 회전)시킨다.
# 2. unstack: 로우를 컬럼으로 피벗시킨다.

In [270]:
# 몇 가지 예제를 통해 위 연산을 좀 더 알아보자. 문자열이 담긴 배열을 로우와 컬럼의 색인으로 하는 작은 DataFrame이 있다.

In [272]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))

In [273]:
# stack 메서드를 사용하면 컬럼이 로우로 피벗되어서 다음과 같은 Series 객체를 반환한다. 

In [274]:
result = data.stack()

In [275]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [276]:
# unstack 메서드를 사용하면 위 계층적 색인을 가진 Series로부터 다시 DataFrame을 얻을 수 있다. 

In [277]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [278]:
# 기본적으로 가장 안쪽에 있는 레벨부터 끄집어내는데(stack도 마찬가지다), 레벨 숫자나 이름을 전달해서 끄집어낼 단계를 지정할 수 있다. 

In [279]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [280]:
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [281]:
# 해당 레벨에 있는 모든 값이 하위그룹에 속하지 않을 경우 unstack을 하게 되면 누락된 데이터가 생길 수 있다. 

In [282]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"])

In [283]:
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"])

In [284]:
data2 = pd.concat([s1, s2], keys=["one", "two"])

In [285]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [286]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [287]:
# stack 메서드는 누락된 데이터를 자동으로 걸러내기 때문에 연산을 쉽게 원상복구할 수 있다. 

In [288]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [290]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [291]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [292]:
# DataFrame을 unstack()할 때 unstack() 레벨은 결과에서 가장 낮은 단계가 된다. 

In [293]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))

In [294]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [295]:
df.unstack("state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [296]:
# stack을 호출할 때 쌓을 축을 이름을 지정할 수 있다. 

In [297]:
df.unstack("state").stack("side")

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [298]:
# 8.3.2 긴 형식에서 넓은 형식으로 피벗하기

In [299]:
# 데이터베이스나 CSV 파일에 여러 개의 시계열 데이터를 저장하는 일반적인 방법은 시간 순서대로 나열하는 것이다. 

In [300]:
# 예제 데이터를 읽어서 시계열 데이터를 다뤄보자.

In [302]:
data = pd.read_csv("examples/macrodata.csv")

In [303]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [304]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name="date")

In [305]:
columns = pd.Index(["realgdp", "infl", "unemp"], name="item")

In [306]:
data = data.reindex(columns=columns)

In [307]:
data.index = periods.to_timestamp("D", "end")

In [308]:
ldata = data.stack().reset_index().rename(columns={0: "value"})

In [309]:
# PeriodIndex는 11장에서 제대로 알아보겠지만 간단히 설명하면 시간 간격을 나타내기 위한 자료형으로, 연도(year)와 분기(quarter) 컬럼을 합친다.

In [310]:
# ldata는 이제 다음과 같다. 

In [311]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [312]:
# 이를 긴 형식이라고 부르며, 여러 시계열이나 둘 이상의 키(예제에서는 date와 item)를 가지고 있는 다른 관측 데이터에서 사용한다. 

In [313]:
# 각 로우는 단일 관측치를 나타낸다.

In [314]:
# MySQL 같은 관계형 데이터베이스는 테이블에 데이터가 추가되거나 삭제되면 item 컬럼에 별개의 값을 넣거나 빼는 방식으로 고정된 스키마(컬럼 이름과 데이터형)에 데이터를 저장한다.

In [315]:
# 위 예에서 date와 item은 관계형 데이터베이스 관점에서 얘기해자면 기본키가 되어 관계 무결성을 제공하며 쉬운 조인 연산과 프로그램에 의한 질의를 가능하게 해준다.

In [316]:
# 물론 단점도 있는데, 길이가 긴 형식으로는 작업이 용이하지 않을 수 있어서 하나의 DataFrame에 date 컬럼의 시간값으로 색인된 개별 item을 컬럼으로 포함시키는 것을 선호할지도 모른다.

In [317]:
# DataFrame의 pivot 메서드가 바로 이런 변형을 지원한다.

In [318]:
pivoted = ldata.pivot("date", "item", "value")

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_22160\832112414.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivoted = ldata.pivot("date", "item", "value")


In [319]:
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [320]:
# pivot 메서드의 처음 두 인자는 로우와 컬럼 색인으로 사용될 컬럼 이름이고 마지막 두 인자는 DataFrame에 채워 넣을 값을 담고 있는 컬럼 이름이다. 

In [321]:
# 한 번에 두 개의 컬럼을 동시에 변형한다고 하자. 

In [322]:
ldata["value2"] = np.random.randn(len(ldata))

In [323]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.286350
1,1959-03-31 23:59:59.999999999,infl,0.000,0.377984
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.753887
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.331286
4,1959-06-30 23:59:59.999999999,infl,2.340,1.349742
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.069877
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.246674
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.011862
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.004812
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.327195


In [324]:
# 마지막 인자를 생략해서 계층적 컬럼을 가지는 DataFrame을 얻을 수 있다. 

In [325]:
pivoted = ldata.pivot("date", "item")

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_22160\3418222154.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  pivoted = ldata.pivot("date", "item")


In [326]:
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.377984  0.286350   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.349742  0.331286   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.011862  0.246674   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.919262  1.327195   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.758363  0.022185   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.753887  
1959-06-30 23:59:59.999999999  0.069877  
1959-09-30 23:59:59.999999999  1.004812  
1959-12-31 23:59:59.999999999 -1.549106  
1960-03-31 23:59:59.999999999 -0.660524

In [327]:
pivoted["value"][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [328]:
# pivot은 단지 set_index를 사용해서 계층적 색인을 만들고 unstack 메서드를 이용해서 형태를 변경하는 단축키 같은 메서드다. 

In [329]:
unstacked = ldata.set_index(["date", "item"]).unstack("item")

In [330]:
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.377984  0.286350   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.349742  0.331286   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.011862  0.246674   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.919262  1.327195   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.758363  0.022185   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.010032  0.862580   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.852965  0.670216   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.753887  
1959-06-30 23:59:59.999999999  0.069877  
1959-09-30 23:59:59.999999999  1.004812  
1959-12-31 23:59:59.999999999 -1.549106  
1960-03-31 23:59:59.999999999 -0.660524  
1960-06-30 23:59:59.999999999  0.050009  
1960-09-30 23:59:59.999999999 -0.955869

In [331]:
# 8.3.3 넓은 형식에서 긴 형식으로 피벗하기

In [332]:
# pivot과 반대되는 연산은 pandas.melt다. 하나의 컬럼을 여러 개의 새로운 DataFrame으로 생성하기보다는 여러 컬럼을 하나로 병합하고 DataFrame을 입력보다 긴 형태로 만들어낸다. 

In [333]:
# 예제를 살펴보자.

In [335]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})

In [336]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [337]:
# "key" 컬럼을 그룹 구분자로 사용할 수 있고 다른 컬럼을 데이터값으로 사용할 수 있다.

In [338]:
# pandas.melt를 사용할 때는 반드시 어떤 컬럼을 그룹 구분자로 사용할 것인지 지정해야 한다. 

In [339]:
# 여기서는 "key"를 그룹 구분자로 지정하자.

In [340]:
melted = pd.melt(df, ["key"])

In [341]:
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [342]:
# pivot을 사용해서 원래 모양으로 되돌릴 수 있다. 

In [343]:
reshaped = melted.pivot("key", "variable", "value")

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_22160\3665212195.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  reshaped = melted.pivot("key", "variable", "value")


In [344]:
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [345]:
# pivot의 결과는 로우 레이블로 사용하던 컬럼에서 색인을 생성하므로 reset_index를 이용해서 데이터를 다시 컬럼으로 돌려놓자.

In [346]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [347]:
# 데이터값으로 사용할 컬럼들의 집합을 지정할 수도 있다.

In [349]:
pd.melt(df, id_vars=["key"], value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [350]:
# pandas.melt는 그룹 구분자 없이도 사용할 수 있다.

In [351]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [352]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [353]:
# 마치며; 지금까지 pandas에서 데이터를 불러오고, 정제하고, 재배열하는 방식을 익혔다. 이제 matplotlib을 이용한 데이터 시각화 단계로 넘어갈 준비가 되었다. 
# 9장을 마친 다음에는 다시 pandas로 돌아와 고급 기법을 다루도록 하겠다.